# Lab | Inferential statistics - ANOVA

Note: The following lab is divided in 2 sections which represent activities 3 and 4.

## Part 1

In this activity, we will look at another example. Your task is to understand the problem and write down all the steps to set up ANOVA. After the next lesson, we will ask you to solve this problem using Python. Here are the steps that you would need to work on:
    - Null hypothesis
    - Alternate hypothesis
    - Level of significance
    - Test statistic
    - P-value
    - F table

### Context

Suppose you are working as an analyst in a microprocessor chip manufacturing plant. You have been given the task of analyzing a plasma etching process with respect to changing Power (in Watts) of the plasma beam. Data was collected and provided to you to conduct statistical analysis and check if changing the power of the plasma beam has any effect on the etching rate by the machine. You will conduct ANOVA and check if there is any difference in the mean etching rate for different levels of power. You can find the data `anova_lab_data.xlsx` file in the `files_for_lab` folder  

- State the null hypothesis

    H<sub>0</sub>: $\mu$<sub>1</sub> = $\mu$<sub>2</sub> = ... = $\mu$<sub>n</sub>

- State the alternate hypothesis

    H<sub>1</sub>: $\mu$<sub>j</sub> != $\mu$<sub>k</sub> for at least one pair of $(j, k)$ where $j$ != $k$

- What is the significance level

    We typically choose a significance level of p = 0.05.

- What are the degrees of freedom of model, error terms, and total DoF

    Degrees of freedom between samples = k - 1, where k = no. of samples
    Degrees of freedom within samples = n - k, where k = no. of samples, n = total no. of observations (total sample size)
    Total degrees of freedom = n - 1, where n = total no. of observations (total sample size)

    **Error within sample**
    $$ SSE = \sum_{i=1}^{k}\sum_{j=1}^{n} (y_{ij} - \overline{y_i}) ^ 2 $$

    **Error across samples**
    $$ SST = \sum_{i=1}^{k} (y_{i} - \overline{y}) ^ 2 $$ 

    **Total error**
    $$ TSS = SSE + SST = \sum_{i=1}^{k}\sum_{j=1}^{n} (y_{ij} - \overline{y}) $$
    
Data was collected randomly and provided to you in the table as shown: [link to the image - Data](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.05/7.05-lab_data.png)


## Part 2

- In this section, use Python to conduct ANOVA.
- What conclusions can you draw from the experiment and why?

We want to confirm or infirm that the mean etching rate is the same across all powers used by the plasma beam.

In [10]:
import numpy as np
import pandas as pd

In [11]:
path = "/home/sabina/Downloads/Ironhack/Unit 7/lab-inferential-statistics-anova/files_for_lab/anova_lab_data.xlsx"
data = pd.read_excel(path)
data.head()

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71


In [12]:
# Check the number of samples in the dataset
samples = list(data["Power "].unique()) # k = 3

In [13]:
# Check the number of observations in the dataset
n = data.shape[0]   # n = 15

In [14]:
# Find the mean of each sample
data.groupby('Power ').agg(np.mean)

,Etching Rate
Power,
160 W,5.792
180 W,6.238
200 W,8.318


In [15]:
# Rename columns
mapper = {"Power ": "power", "Etching Rate": "etching_rate"}
data.rename(mapper=mapper, axis=1, inplace=True)
data.columns

Index(['power', 'etching_rate'], dtype='object')

In [16]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('etching_rate ~ power ',data=data).fit()
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
power,2.0,18.176653,9.088327,36.878955,0.000008
Residual,12.0,2.957240,0.246437,NaN,NaN


The p-value << 0.05, so we can reject the null hypothesis, meaning that the change in plasma beam power leads to a change in mean for at least one of the pairs. To find out in which pair(s) the change in plasma power leads to a change in the etching rate, we need to run t-tests for each pair:

In [22]:
from scipy.stats import ttest_ind

for sample in samples:
    elem = samples.index(sample)

    if elem < len(samples) - 1:
        next_sample = samples[elem + 1]
    else:
        sample = samples[0]
        next_sample = samples[-1]

    statistic, p_value = ttest_ind(data[data["power"] == sample]["etching_rate"], \
        data[data["power"] == next_sample]["etching_rate"], equal_var=True)

    p_value = round(p_value, 4)

    print("For the", sample, ",", next_sample, "pair we have a p-value of", p_value)

For the 160 W , 180 W pair we have a p-value of 0.1016
For the 180 W , 200 W pair we have a p-value of 0.0004
For the 160 W , 200 W pair we have a p-value of 0.0001


We can see that we're able to reject the null hypothesis for the (160W, 180W) pair, meaning that an increase in the plasma beam power from 160W to 180W will increase the average etching rate, whereas a further increase from 180W to 200W will not do so. However, what's strange is that the p-value for the (160W, 200W) pair suggests the null hypothesis is true. Why is that occurring?